In [1]:
!pip install beautifulsoup4

In [2]:
!pip install lxml

In [3]:
!pip install nltk

In [84]:
!pip install googletrans

In [112]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [113]:
def getHTMLContent(url): 
  try:
    response = get(url)
    html = BeautifulSoup(response.text, 'html.parser')
  except Exception as e:
    html = None
  return html

In [114]:
hospitals = {
  'rajavithi': 'https://www.honestdocs.co/hospitals/rajavithi-hospital',
  'bumrungrad': 'https://www.honestdocs.co/hospitals/bumrungrad-hospital',
  'lerdsin': 'https://www.honestdocs.co/hospitals/lerdsin-hospital'
}
for key, url in hospitals.items():
  ratings = []
  reviews = []
  page = 1
  while True:
    time.sleep(1)
    full_url = url+'?page='+str(page)
    html = getHTMLContent(full_url)
    if html is None:
      break
    comments = html.find_all('div', class_='comments__container')
    if not comments :
      break
    for comment in comments:
      rating = comment.find("span", class_="star-rating")["data-score"]
      if(int(rating) == 3):
        continue
      elif (int(rating) <= 2):
        ratings.append("0")
      else:
        ratings.append("1")
      review = comment.find("div", class_="comments__content").p.text
      reviews.append(review)
    page += 1
  df = pd.DataFrame({"reviews": reviews,
                   "ratings": ratings})
  df.to_csv(key+".csv", index=False)
  time.sleep(5)

KeyboardInterrupt: 

In [115]:
rajavithi_df = pd.read_csv("rajavithi.csv")
lerdsin_df = pd.read_csv("lerdsin.csv")
bumrungrad_df = pd.read_csv("bumrungrad.csv")

In [116]:
rajavithi_df.head()

,reviews,ratings
0,บรรยากาศโรงพยาบาลสวยงามมีแอร์เย็นสบ...,1
1,มีศูนย์โรงผิวหนังเฉพาะทางที่ดีมากๆ​ คุณหมอทุกค...,1
2,หมอและพยาบาลบริการดีครับ สุขภาพเรียบร้อยให้การ...,1
3,การให้บริการของพยาบาลและคุณหมอดีมากๆค่ะ โรงพยา...,1
4,ที่ โรงพยาบาลราชวิถี ให้การตรวจรักษาและให้บริก...,1


In [117]:
lerdsin_df.head()

,reviews,ratings
0,จำไม่ได้ว่าปีไหน น่าจะ ช่วงปลายปี 57 หรือ 58 ต...,1
1,เคยใช้บริการตอนขาหัก โดยใช้สิทธิ์ประกันสังคม ไ...,1
2,รอคิวตรวจนานมากค่ะ มาตั้งแต่ 6 โมงเช้า กว่าจะไ...,1
3,มาครั้งแรก ประทับใจมากคับ บริการเหมือน รพ เอกช...,1
4,บริการดีครับ แต่คนเยอะมากๆเลย ที่ใช้บริการคือม...,1


In [118]:
bumrungrad_df.head()

,reviews,ratings
0,เป็นโรงพยาบาลที่ดีมากมารยาทการดี มีความเอาใจใส...,1
1,รู้สึกได้รับความรู้จากบทความนี้มากๆครับ และสาม...,1
2,รู้สึกได้รับความรู้จากบทความนี้มากๆครับ และสาม...,1
3,โรงพยาบาลบำรุงราษฎร์ ให้บริการดีเยี่ยม พยาบาล ...,1
4,รู้สึกได้รับความรู้จากบทความนี้มากๆครับ และสาม...,1


In [119]:
from googletrans import Translator

In [124]:
translator = Translator()

In [127]:
"""get english translated column.
        Parameters
        ----------
        reviews : string
             review of customer in hospital
"""
def apply_translator(reviews):
    reviews_en = []
    for row in reviews:
        try:
            reviews_en.append(translator.translate(row, src="th",dest="en").text)
        except Exception as e:
            reviews_en.append("")
            print(e)
    return reviews_en

In [128]:
rajavithi_df["reviews_en"] = apply_translator(rajavithi_df["reviews"])
rajavithi_df.head()

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [111]:
lerdsin_df["reviews_en"] = apply_translator(lerdsin_df["reviews"])
lerdsin_df.head()

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [41]:
bumrungrad_df["reviews_en"] = apply_translator(bumrungrad_df["reviews"])
bumrungrad_df.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [129]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [130]:
"""get removed stopword column.
        Parameters
        ----------
        reviews_en : str
            comment in english language
"""
def apply_stopwords(reviews_en):
    tokenize_words_list = []
    for row in reviews_en:
        words = nltk.tokenize.word_tokenize(row)
        tokenize_words = [word.lower() for word in words not in nltk.corpus.stopwords.words("english")]
        tokenize_words_list.append(tokenize_words)
    return tokenize_words_list

In [ ]:
rajavithi_df["word_list"] = apply_stopwords(rajavithi_df["reviews_en"])
rajavithi_df.head()

In [ ]:
lerdsin_df["word_list"] = apply_stopwords(lerdsin_df["reviews_en"])
lerdsin_df.head()

In [ ]:
bumrungrad_df["word_list"] = apply_stopwords(bumrungrad_df["reviews_en"])
bumrungrad_df.head()

In [134]:
from nltk.stem import PorterStemmer

"""get stemming words column.
        Parameters
        ----------
        word_list : list of list of string
             list of list of word
"""
def apply_stemming(word_list):
    stemming_list = []
    porter = PorterStemmer()
    for row in word_list:
        stemming_words = [porter.stem(word) for word in word_list]
        stemming_list.append(stemming_words)
    return stemming_list

In [ ]:
rajavithi_df["stemming_list"] = apply_stemming(rajavithi_df["word_list"])
rajavithi_df.head()

In [ ]:
lerdsin_df["stemming_list"] = apply_stemming(lerdsin_df["word_list"])
lerdsin_df.head()

In [ ]:
bumrungrad_df["stemming_list"] = apply_stemming(bumrungrad_df["word_list"])
bumrungrad_df.head()